# 1. 데이터 불러오기

In [ ]:
import pandas as pd 

df = pd.read_csv(
    "./data/appreply2.csv",
    index_col=0
)
df.head(2)

In [ ]:
# 장바구니 분석 과정
## 1. 지지도, 신뢰도, 향상도 지표 계산하기 (aprior 알고리즘)
## 1) 데이터 준비 1
## [[문장 1의 토크나이징 결과], [문장 2의 토크나이징 결과], ...]
## 2) 데이터 준비 2
## 예시: [[오늘], [하루, 맑음], ...]
## 데이터 프레임
##       오늘 하루 맑음
## 문장1 True False ...
## 문장2
## 문장3

# 2. 데이터 준비하기 1

In [ ]:
# 워드 클라우드 그릴 때 [문장 1의 토크나이징 결과, 문장 2의 토크나이징 결과, ....]
# [[문장 1의 토크나이징 결과], [문장 2의 토크나이징 결과], ...]

In [ ]:
import re 
from konlpy.tag import Okt 

okt = Okt() 

In [ ]:
# 반복문
word_list = []
stopwords = []

# 패턴: [^0-9가-힣a-zA-Z\s]
for sent in df["text"]: # sent: 문장
    print("STEP1. 문장을 전처리합니다.")
    # 리뷰 텍스트 전처리
    clean_sent = re.sub(r"[^0-9가-힣a-zA-Z\s]", "", sent)
    print("STEP2. 문장을 형태소분석기로 토크나이징합니다.")
    # 토크나이징(형태소 분석기: Okt)
    result = okt.pos(clean_sent)
    # result를 하나씩 뽑는다. 
    print("STEP3. 새로운 리스트를 만듭니다.")
    sub_list = []
    print("\t탐색을 시작합니다.")
    for res in result: # res : (단어, 품사)
        word = res[0]
        pos = res[1]
        # word가 stopwords에 있으면 건너뛰기
        if word in stopwords:
            print("\t건너뛰어!", res)
            continue

        # sub_list 만들기: 조건 pos == "Noun" and len(word) > 1
        if pos == "Noun" and len(word) > 1:
            sub_list.append(word)
        # print(f"[RES] {res} [WORD] {word} [POS] {pos}")
    print("\t탐색을 종료합니다.")
    print(f"[SUB LIST] {sub_list}")
    # print(f"[ORIGINAL] {sent}")
    # print(f"[CLEAN] {clean_sent}")
    # print(f"[TOKENIZING] {result}") # 반복문으로 하나씩 확인해야 한다.
    print(f"STEP4. word_list에 sub_list를 추가합니다.")
    word_list.append(sub_list)
    print(f"\t[WORD LIST] {word_list}")
    print("="*100)

In [ ]:
print(len(word_list))

In [ ]:
for x in word_list:
    print(x)
    print("="*30)

# 3. 데이터 준비하기 2

In [ ]:
# uv add mlxtend

In [ ]:
from mlxtend.preprocessing import TransactionEncoder 

te = TransactionEncoder()
te_arr = te.fit(word_list).transform(word_list)
te_arr

In [ ]:
import pandas as pd 

# 문장에 각각의 단어들이 있는지 없는지를 True / False
df = pd.DataFrame(te_arr, columns=te.columns_)
df

In [ ]:
# 단어별로 출현한 횟수
df.sum()

# 4. 장바구니 분석

In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules 

# support는 전체 문장 중에 itemset이 등장한 문장의 비율
# min_support : support가 min_support 이상인것만 보여주세요
# max_len : max_len개의 조합까지 보여주세요
frequent_itemsets = apriori(df, min_support=0.02, use_colnames=True, max_len=2)
frequent_itemsets

In [ ]:
# metric이 min_threshold 이상인 것만 보여주세요.
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)
rules[["antecedents", "consequents", "support", "confidence", "lift"]]

In [ ]:
# frozenset({검색})이라고 되어있는 분들도 똑같으니 그대로 진행하시면 됩니다. 

In [ ]:
# 지지도: support(A-B) = support(B-A) = P(A∩B) = 전체 문장 중 A,B가 동시에 들어간 문장의 비율
# 신뢰도: confidence(A-B) = P(B|A), confidence(B-A) = P(A|B)
# 향상도: lift(A-B) = confidence(A-B)/P(A), lift(B-A) = confidence(B-A)/P(B)

In [ ]:
# 표현: antecedents - consequents
## 예시 1: 가게 - 개선
## support: 전체 문장에서 "가게", "개선"이 모두 등장한 문장의 비율
## -> 1000문장 중에 29문장은 "가게", "개선"이 같이 등장했다.
## confidence: "가게"가 등장한 문장 중에서 "개선", "가게"가 동시에 등장한 문장의 비율
## -> "가게" 문장이 예를 들어 100라면 100개 중 14개 정도는 "개선"이 들어가 있더라.
## lift: "가게"가 등장했을 때와 "가게" 문장에서 "개선"이 들어가 있을 때를 비교 (우연이 아니다.)
## -> "가게"가 등장할 확률보다 "가게" 문장들에서 "개선"이 들어가 있는 확률이 1.36배 높더라.

## 예시 2: 개선 - 가게
## support: 전체 문장에서 "가게", "개선"이 모두 등장한 문장의 비율
## -> 1000문장 중에 29문장은 "가게", "개선"이 같이 등장했다.
## confidence: "개선"이 등장한 문장 중에서 "개선", "가게"가 동시에 등장한 문장의 비율
## -> "개선" 문장이 예를 들어 100라면 100개 중 28개 정도는 "가게"이 들어가 있더라.
## lift: "개선"이 등장했을 때와 "개선" 문장에서 "가게"이 들어가 있을 때를 비교 (우연이 아니다.)
## -> "개선"이 등장할 확률보다 "개선" 문장들에서 "가게"이 들어가 있는 확률이 1.36배 높더라.

# 5. 시각화(히트맵)

In [ ]:
rules[["antecedents", "consequents", "support", "confidence", "lift"]]

In [ ]:
# pivot_table
pivot_data = rules.head(20).pivot_table(
    index="antecedents",        # 행
    columns="consequents",      # 열
    values="lift",              # 기준
    fill_value=0                # 매칭되지 않는 것은 이걸로 채워라.
)
pivot_data

In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns
import koreanize_matplotlib

plt.figure(figsize=(10,10))
sns.heatmap(pivot_data, annot=True, cmap="YlGnBu", fmt=".2f", linewidths=0.3, square=True)
plt.title("연관성 분석 시각화(Lift 기준)")
plt.xlabel("Consequents")
plt.ylabel("Antecedents")
plt.show()

# 6. 시각화(네트워크 분석)

In [ ]:
# uv add networkx

In [ ]:
rules[["antecedents","consequents","support","confidence", "lift"]].head(2)

In [ ]:
# iterrows 이해하기
# 데이터프레임에서 하나의 행씩 추출 (인덱스, 열 데이터)
sample_data = rules[["antecedents", "consequents"]].head(2)
for x in sample_data.iterrows():
    print(f"x의 요소 개수: {len(x)}")
    print(x[0])
    print("-"*50)
    print(x[1])
    print("="*50)

In [ ]:
# .join() 이해하기
test = ["오늘", "하루", "맑음"]
print("-".join(test)) # ,를 다른 문자들로 바꿔보세요.

In [ ]:
rules

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt 
import koreanize_matplotlib

In [ ]:
## 보여주고 싶은 데이터 설정하기(여러분이 설정하세요.)
my_rules = rules.sort_values(by=["lift"], ascending=False).head(50)

# 1. 그래프 생성
G = nx.Graph()

# 2. 엣지 추가
for _ , col_data in my_rules.iterrows():
    # 1) 단어 추출
    print(f"[BEFORE] {col_data['antecedents']}, {col_data['consequents']}")
    antecedent = ",".join(col_data["antecedents"])
    consequent = ",".join(col_data["consequents"])
    print(f"[AFTER] {antecedent}, {consequent}")
    print("="*100)

    # 2) 지표 추출
    weight = col_data["lift"]

    # 3) 그래프에 정보 추가
    G.add_edge(antecedent, consequent, weight=weight)


In [ ]:
G.edges()

In [ ]:
G["가입"]["회원"]

In [ ]:
# 3. 노드 배치
# position = nx.kamada_kawai_layout(G, scale=0.5)
# k를 조절하면 노드간 거리를 조절할 수 있습니다.
position = nx.spring_layout(G, k=0.9, seed=15)

# 4. 가중치 추출
scale = 0.3 ## 선의 굵기가 너무 굵다면 사이즈를 줄일 수 있습니다.
edge_weights = [G[u][v]["weight"]*scale for u, v in G.edges()]
print(edge_weights)

# 5. 그리기
plt.figure(figsize=(10,10))
nx.draw_networkx_nodes(G, position, node_color="lightblue", node_size=1000)
nx.draw_networkx_edges(G, position, edge_color="gray", width=edge_weights)
nx.draw_networkx_labels(G, position, font_size=10, font_family="Malgun Gothic")
plt.title("단어 간 연관규칙 기반 네트워크 그래프(Lift 기준)")
plt.axis("off")
plt.show()